In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from pathlib import Path


In [ ]:

count = 0

In [ ]:
def store_data(data,topic):
    global count
    cwd = os.getcwd()
    cwd = os.path.join(cwd,'train_data')
    complete_dir = os.path.join(cwd, topic)
    print(complete_dir)
    if not os.path.exists(complete_dir):
        os.makedirs(complete_dir)
    file_path = os.path.join(complete_dir,str(count) + '.txt')
    print(file_path)
    with open(file_path, 'w',encoding="utf-8") as f:
        f.write(data)
        f.close()
    count += 1


In [ ]:
def basic_handding(data_context):
    data_context =re.sub(r'\n+', '\n', data_context).strip()
    data_context =re.sub(r'{6}.+\n', '', data_context).strip()
    data_context =re.sub(r'https://.+\n', '', data_context).strip()
    return data_context

In [ ]:
list_topic = ['covid-19']


In [ ]:
import threading
threadLock = threading.Lock()

In [ ]:


class FirstThread(threading.Thread):
  def __init__(self,thread_name,start_index,end_index,topic):
    threading.Thread.__init__(self)
    self.start_index = start_index
    self.end_index = end_index
    self.topic = topic
    self.thread_name = thread_name
  def run(self):

    global count
    for i in range(self.start_index,self.end_index):
        url = 'https://theconversation.com/us/{}/articles?page={}'.format(self.topic,i)
        headers = {
        'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
        }
        response = requests.request("GET", url, headers=headers)
        search_page_data = BeautifulSoup(response.text, 'html.parser')
        
        list_articles = search_page_data.find_all('article',attrs={'class':'clearfix placed analysis published'})

        
        for article in list_articles:
            article_url = 'https://theconversation.com'+article.find('a',attrs={'class':'article-link'})['href']
            print(article_url)
            headers_article_url = {
            'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
            }
            response_article = requests.request("GET", article_url, headers=headers_article_url)
            data_article = BeautifulSoup(response_article.text, 'html.parser')
            content_data  =data_article.find_all('div',attrs={'class':'grid-ten large-grid-nine grid-last content-body content entry-content instapaper_body inline-promos'})
            for content in content_data:
                threadLock.acquire()
                print(basic_handding(content.text))
                store_data(basic_handding(content.text),topic)
                print('#'*200)
                threadLock.release()

    


In [ ]:
count = 0
for topic in list_topic:
  thread1 = FirstThread("Thread_1", 1,15,topic)
  thread2 = FirstThread("Thread_2", 15,30,topic)
  thread3 = FirstThread("Thread_3", 30,45,topic)
  thread4 = FirstThread("Thread_4", 45,60,topic)
  thread5 = FirstThread("Thread_5", 60,75,topic)
  thread6 = FirstThread("Thread_6", 75,90,topic)

  thread2.start()
  thread3.start()
  thread4.start()
  thread5.start()
  thread6.start()


  threads = [thread1, thread2, thread3, thread4, thread5, thread6]

  for t in threads:
    t.join()
